In [29]:
# import lxml
from lxml import etree
from lxml import objectify

In [54]:
import os
import json

In [89]:
class Validator:
    
    def __init__(self, scenario=False):
        self.__load_config_file()
        
        if scenario == "dictionary":
            schema_name = "dictionary_schema_path"
        elif scenario == "drama":
            schema_name = "drame_schema_path"
        else:
            schema_name = "basic_schema_path"
        self.schema = self.load_rng_schema(self.config[schema_name])
        
    @staticmethod
    def read_json(path):
        with open(path) as file:
            data = json.load(file)
        return data
        
    def __load_config_file(self):
        path = os.path.join('data', 'config.json') 
        self.config = self.read_json(path)

    def __check_file_extension(self, path, extension):
        fname, fextension = os.path.splitext(path)
        if fextension != extension:
            raise ValueError("Wrong file extension")
        return True

    def load_rng_schema(self, schema_path):
        if self.__check_file_extension(schema_path, ".rng"):
            schema_doc = etree.parse(schema_path)
            self.schema = etree.RelaxNG(schema_doc)

    def validate(self, xml_path):
        xml_doc = etree.parse(xml_path)
        result = self.schema.validate(xml_doc)
        return result



# Tests

In [99]:
def basic_test(v, dir_path):
    for filename in os.listdir(dir_path):
        if filename != ".DS_Store":
            file_path = os.path.join(dir_path, filename)
            try:
                res = v.validate(file_path)
                print(output.format(res, filename))
            except:
                print(output.format('Error', filename))

### test 1

In [100]:
v = Validator()
dir_path = os.path.join('data', 'teis')
output = 'RESULT {} FOR file {}'
basic_test(v, dir_path)

RESULT Error FOR file afinogenov-mashenka.xml
RESULT Error FOR file afinogenov_changed_namespace.xml
RESULT Error FOR file afinogenov_extra_tag.xml
RESULT Error FOR file afinogenov_new_namespace.xml
RESULT Error FOR file afinogenov_new_tag.xml
RESULT Error FOR file afinogenov_no_schema.xml
RESULT Error FOR file afinogenov_no_start.xml


### test 2

In [101]:
v = Validator(scenario="drama")
basic_test(v, dir_path)

RESULT Error FOR file afinogenov-mashenka.xml
RESULT Error FOR file afinogenov_changed_namespace.xml
RESULT Error FOR file afinogenov_extra_tag.xml
RESULT Error FOR file afinogenov_new_namespace.xml
RESULT Error FOR file afinogenov_new_tag.xml
RESULT Error FOR file afinogenov_no_schema.xml
RESULT Error FOR file afinogenov_no_start.xml


### test 3

In [102]:
v = Validator()
schema_path = "data/schemas/schema_dracor.rng"
v.load_rng_schema(schema_path)
basic_test(v, dir_path)

RESULT True FOR file afinogenov-mashenka.xml
RESULT False FOR file afinogenov_changed_namespace.xml
RESULT Error FOR file afinogenov_extra_tag.xml
RESULT Error FOR file afinogenov_new_namespace.xml
RESULT Error FOR file afinogenov_new_tag.xml
RESULT True FOR file afinogenov_no_schema.xml
RESULT True FOR file afinogenov_no_start.xml
